In [1]:
import torch                                  #Imports 
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class evaluation():   #Evaluation Class
    
 def __init__(self,name ): 
     self.name=name
 


In [155]:
class Chovek():                                   #Evaluation Class 
  def __init__(self,name ):                               # which parameters should it receive 
    
   self.name = name

  def hi(self):
    
    message = f'hi, az sum {self.name}'
    return message


In [156]:
ivan = Chovek(name='Ivan')

ivan.hi()

'hi, az sum Ivan'

In [157]:
train_loader = torch.utils.data.DataLoader(                             #Train the Data and Testloaders and normalize the inputs
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [158]:
n_epochs = 3                       #Definition of Hyper Parameters
batch_size_train = 60
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [159]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [160]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [ ]:
import matplotlib.pyplot as plt               # Display six images 

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [162]:
import torch.nn as nn                            #Imports of Neural Network 
import torch.nn.functional as F
import torch.optim as optim

In [163]:
class Net(nn.Module):                                        #Neural Network
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [164]:
network = Net()                                                                     #Initialization of Neural Network and of the Optimizer
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [165]:
train_losses = []                                             # We'll also keep track of the progress with some printouts. In order to create a nice training curve later on we also create two lists for saving training and testing losses. On the x-axis we want to display the number of training examples the network has seen during training. 
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [166]:
import os

def train(epoch):
                              # Training of the Neural Network
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), str(os.getcwd()) + '/results/model.pth')
      torch.save(optimizer.state_dict(), str(os.getcwd()) + '/results/optimizer.pth')

In [167]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [168]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

C:\Users\ivank\AppData\Local\Temp/ipykernel_24104/2161508242.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.3096, Accuracy: 924/10000 (9%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311668
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.301118
Train Epoch: 1 [1200/60000 (2%)]	Loss: 2.288145
Train Epoch: 1 [1800/60000 (3%)]	Loss: 2.247380
Train Epoch: 1 [2400/60000 (4%)]	Loss: 2.253007
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.238811
Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.226169
Train Epoch: 1 [4200/60000 (7%)]	Loss: 2.167308
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.093194
Train Epoch: 1 [5400/60000 (9%)]	Loss: 2.108451
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.913602
Train Epoch: 1 [6600/60000 (11%)]	Loss: 1.685405
Train Epoch: 1 [7200/60000 (12%)]	Loss: 1.731126
Train Epoch: 1 [7800/60000 (13%)]	Loss: 1.769907
Train Epoch: 1 [8400/60000 (14%)]	Loss: 1.515585
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.493429
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.193665
Train Epoch: 1 [10200/60000 (17%)]	Loss: 1.502354
Train Epoch: 1 [10800/60000 (18%)]	Loss: 0.964918
Train Epoch: 1 [11400/600

In [3]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
fig

NameError: name 'train_counter' is not defined

<Figure size 432x288 with 0 Axes>

In [172]:
len(train_loader)

1000

In [173]:
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

In [176]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(                             #Train the Data and Testloaders and normalize the inputs
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])))
    train_preds = get_all_preds(network, prediction_loader)

C:\Users\ivank\AppData\Local\Temp/ipykernel_24104/2161508242.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [177]:
print(train_preds)

tensor([[-1.0408e+01, -1.3709e+01, -1.6188e+01,  ..., -1.7318e+01,
         -1.0667e+01, -9.2006e+00],
        [-2.0146e-05, -1.8660e+01, -1.1309e+01,  ..., -1.6409e+01,
         -1.4475e+01, -1.4082e+01],
        [-1.8827e+01, -1.6302e+01, -1.2423e+01,  ..., -1.2560e+01,
         -1.2810e+01, -3.8339e+00],
        ...,
        [-1.4277e+01, -2.0713e+01, -2.5359e+01,  ..., -2.1848e+01,
         -1.1188e+01, -8.7156e+00],
        [-7.2641e+00, -1.1322e+01, -9.4318e+00,  ..., -1.7674e+01,
         -9.4335e+00, -1.2322e+01],
        [-5.6276e+00, -9.3212e+00, -7.4222e+00,  ..., -7.9902e+00,
         -2.2738e-02, -4.2860e+00]])


In [180]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [181]:
preds_correct = get_num_correct(train_preds, train_loader.)

print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(train_loader))

AttributeError: 'DataLoader' object has no attribute 'targets'